In [ ]:
#Dunkelflaute absolute percentage change when compared to historical
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib as mpl

proj = ccrs.PlateCarree()
fig, axes = plt.subplots(
    nrows=1, ncols=2, figsize=(14, 4.5),
    subplot_kw=dict(projection=proj),
    layout='compressed',
)
discrete_cmap = mpl.cm.get_cmap("bwr", 11)
discrete_cmap.set_bad(color=(0, 0, 0, 0))   

file_1=xr.open_dataset("/work/gg0302/g260190/rsds_analysis/data/Dunkelflaute/Timesum/dunkelflaute_historical.nc")
file_2=xr.open_dataset("/work/gg0302/g260190/rsds_analysis/data/Dunkelflaute/Timesum/dunkelflaute_GWL2K.nc")
file_3=xr.open_dataset("/work/gg0302/g260190/rsds_analysis/data/Dunkelflaute/Timesum/dunkelflaute_GWL3K.nc")

GWL2K=file_2["Dunkelflaute"].values-file_1["Dunkelflaute"].values
GWL3K=file_3["Dunkelflaute"].values-file_1["Dunkelflaute"].values
if np.nanmin(GWL2K)<np.nanmin(GWL3K):
    v_min=np.nanmin(GWL2K)
else:
    v_min=np.nanmin(GWL3K)

if np.nanmax(GWL2K)>np.nanmax(GWL3K):
    v_max=np.nanmax(GWL2K)
else:
    v_max=np.nanmax(GWL3K)
v_min=v_min*100
v_max=v_max*100

if abs(v_min)>abs(v_max):
    v_max=abs(v_min)
else:
    v_min=-v_max

files=[GWL2K,GWL3K]
scenarios=["GWL2K","GWL3K"]
for ax, file, scenario in zip(axes, files,scenarios):
    cf_flaute=file*100
    lat = file_1['lat'].values
    lon = file_1['lon'].values 

    cf_masked = np.squeeze(np.ma.masked_equal(cf_flaute, 0))

    pm = ax.pcolormesh(lon, lat, cf_masked,
                    cmap=discrete_cmap,vmin=v_min,vmax=v_max,
                    shading='auto',
                    transform=proj)

    ax.coastlines(linewidth=0.6)
    ax.add_feature(cfeature.BORDERS, linewidth=0.3)
    ax.set_title(scenario, fontsize=12)

cbar = fig.colorbar(pm, ax=axes.ravel().tolist(), orientation='horizontal', pad=0.05, shrink=0.8, label='Absolute % change to time steps with CF < 0.2')
plt.savefig("Dunkelflaute_Summary/dunkelflaute_changes.pdf", bbox_inches="tight", format="pdf", dpi=300)
plt.show()
print(f"Mean value historical: {np.nanmean(file_1['Dunkelflaute'].values)*100}%")
print(f"Mean value GWL2K: {file_2['Dunkelflaute'].mean().item()*100}%")
print(f"Mean value GWL3K: {file_3['Dunkelflaute'].mean().item()*100}%")
print(f"Mean difference GWL2K: {np.nanmean(GWL2K)*100}%")
print(f"Mean difference GWL3K: {np.nanmean(GWL3K)*100}%")